In [2]:
# Setup the Jupyter version of Dash
# Configure the necessary Python module imports
import base64

import dash_leaflet as dl

# Configure the plotting routines
import pandas as pd
import plotly.express as px
from dash import dash_table, dcc, html
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash

# Import the AnimalShelter class from the grazioso.crud module
from grazioso.crud import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# Connect to MongoDB using your AnimalShelter class
username = "aacuser"
password = "SNHU1234"  # Updated to match your test_crud_notebook.ipynb
host = "localhost"
port = 27017
db = "AAC"
col = "animals"


# Create an instance of the AnimalShelter class with all required parameters
shelter = AnimalShelter(username, password, host, port, db, col)

# Define the filters for the different rescue types
filters = {
    "all": {},
    "water": {"animal_type": "Dog", 
              "breed": {"$in": [
                  "Labrador Retriever Mix",
                  "Chesapeake Bay Retriever",
                  "Newfoundland"
                ]
            },
              "sex_upon_outcome": "Intact Female",
              "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
             },
    "mountain": {"animal_type": "Dog", 
                 "breed": {"$in": [
                     "German Shepherd",
                     "Alaskan Malamute",
                     "Old English Sheepdog",
                     "Siberian Husky",
                     "Rottweiler"
                    ]
                },
                 "sex_upon_outcome": "Intact Male",
                 "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
                },
    "disaster": {"animal_type": "Dog", 
                 "breed": {"$in": [
                     "Doberman Pinscher",
                     "German Shepherd",
                     "Golden Retriever",
                     "Bloodhound",
                     "Rottweiler"
                    ]
                },
                 "sex_upon_outcome": "Intact Male",
                 "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
                }
}
# Initial data load - all animals
df = pd.DataFrame.from_records(shelter.read({})) # type: ignore

# MongoDB v5+ returns the '_id' column with an ObjectID type
# We need to drop it to avoid issues with the data_table
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)

# Load the Grazioso Salvare logo
# You would need to replace this with the actual path to the logo
# For now, we'll use a placeholder
image_filename = 'grazioso_salvare_logo.png'  # Replace with your logo path
try:
    encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode('ascii')
except:  # noqa: E722
    # If the image file doesn't exist, we'll just skip it
    encoded_image = None


#########################
# Dashboard Layout / View
#########################
app: JupyterDash = JupyterDash(__name__)

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    
    # Header with logo and title
    html.Div([
        html.Div([
            # Display logo if available
            html.Img(
                src=f'data:image/png;base64,{encoded_image}' if encoded_image else None,
                style={'height': '100px', 'margin-right': '20px'}
            ) if encoded_image else None,
            
            html.Div([
                html.H1('Grazioso Salvare', style={'margin-bottom': '0px'}),
                html.H3('Search and Rescue Dog Finder', style={'margin-top': '0px'})
            ])
        ], style={'display': 'flex', 'align-items': 'center', 'justify-content': 'center'}),
    ]),
    
    html.Hr(),
    
    # Filter options
    html.Div([
        html.Label('Filter by Rescue Type:'),
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'All Dogs', 'value': 'all'},
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
                {'label': 'Disaster Rescue or Individual Tracking', 'value': 'disaster'}
            ],
            value='all',
            inline=True
        )
    ], style={'margin': '10px', 'text-align': 'center'}),
    
    html.Hr(),
    
    # Data table
    html.Div([
        dash_table.DataTable(
            id='datatable-id',
            columns=[
                {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
            ],
            data=df.to_dict('records'),
            # Make the table user-friendly with these features:
            editable=False,              # Don't allow editing
            filter_action="native",      # Allow filtering of data by user
            sort_action="native",        # Allow sorting of data by user
            sort_mode="multi",           # Sort across multiple columns
            column_selectable="single",  # Allow selecting columns
            row_selectable="single",     # Allow selecting rows
            row_deletable=False,         # Prevent row deletion
            selected_columns=[],         # Initially, no columns are selected
            selected_rows=[],            # Initially, no rows are selected
            page_action="native",        # All data is passed to the table up-front
            page_current=0,              # Start on first page
            page_size=10,                # Show 10 rows per page
            style_cell={                 # Style cells
                'font-size': '12px',
                'text-align': 'left'
            },
            style_data={                 # Style data cells
                'whiteSpace': 'normal',
                'height': 'auto',
            },
            style_header={               # Style header cells
                'backgroundColor': 'rgb(30, 30, 30)',
                'color': 'white',
                'fontWeight': 'bold'
            },
            style_data_conditional=[     # Highlight selected rows
                {
                    'if': {'row_index': 'odd'},
                    'backgroundColor': 'rgb(248, 248, 248)'
                }
            ]
        )
    ], style={'margin': '10px'}),
    
    html.Br(),
    html.Hr(),
    
    # Map and chart container
    html.Div([
        # Map container
        html.Div([
            html.H3('Location of Selected Animal'),
            html.Div(id='map-id')
        ], style={'width': '49%', 'display': 'inline-block', 'vertical-align': 'top'}),
        
        # Pie chart container
        html.Div([
            html.H3('Outcomes by Animal Type'),
            dcc.Graph(id='pie-chart')
        ], style={'width': '49%', 'display': 'inline-block'})
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# Callback to update the data table based on the selected filter
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_table(filter_value):
    # Get the filter based on the selected value
    selected_filter = filters[filter_value]
    # Query the database with the selected filter
    results = shelter.read(selected_filter)
    # Convert results to DataFrame
    df_filtered = pd.DataFrame.from_records(results)
    # Drop the _id column if it exists
    if '_id' in df_filtered.columns:
        df_filtered.drop(columns=['_id'], inplace=True)
    # Return the filtered data
    return df_filtered.to_dict('records')

# Callback to highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns'),
     Input('datatable-id', 'selected_rows')]
)
def update_styles(selected_columns, selected_rows):
    styles = []
    
    # Highlight selected columns
    if selected_columns:
        styles += [{
            'if': {'column_id': i},
            'background_color': '#D2F3FF'
        } for i in selected_columns]
    
    # Highlight selected rows
    if selected_rows:
        styles += [{
            'if': {'row_index': i},
            'background_color': '#FFFFCC'
        } for i in selected_rows]
        
    return styles

# Callback to update the geo-location chart for the selected data entry
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, selected_rows):
    dff = pd.DataFrame.from_dict(viewData)
    
    # If no row is selected, default to first row
    if selected_rows is None or len(selected_rows) == 0:
        row = 0
    else:
        row = selected_rows[0]
    
    # Check if the location columns exist and contain valid data
    if 'location_lat' in dff.columns and 'location_long' in dff.columns and len(dff) > 0:
        lat = dff.iloc[row]['location_lat']
        lon = dff.iloc[row]['location_long']
        
        # Default to Austin TX coordinates if no location data
        if pd.isna(lat) or pd.isna(lon):
            lat, lon = 30.75, -97.48
        
        # Get animal info for tooltip and popup
        breed = dff.iloc[row]['breed'] if 'breed' in dff.columns else "Unknown"
        name = dff.iloc[row]['name'] if 'name' in dff.columns else "Unnamed"
        if pd.isna(name) or name == "":
            name = "Unnamed"
        
        return [
            dl.Map(style={'width': '100%', 'height': '500px'},
                  center=[lat, lon], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
                # Marker with tool tip and popup
                dl.Marker(position=[lat, lon],
                         children=[
                             dl.Tooltip(breed),
                             dl.Popup([
                                 html.H3("Animal Details"),
                                 html.P(f"Name: {name}"),
                                 html.P(f"Breed: {breed}"),
                                 html.P(f"Type: {dff.iloc[row]['animal_type'] if 'animal_type' in dff.columns else 'Unknown'}")
                             ])
                         ])
            ])
        ]
    else:
        # Return empty map centered on Austin TX if no data
        return [
            dl.Map(style={'width': '100%', 'height': '500px'},
                  center=[30.75, -97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id")
            ])
        ]

# Callback to update the pie chart showing outcome distribution
@app.callback(
    Output('pie-chart', 'figure'),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_pie_chart(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    
    if 'animal_type' in dff.columns and len(dff) > 0:
        # Create pie chart showing distribution by animal type
        fig = px.pie(dff, names='animal_type', title='Distribution by Animal Type')
        fig.update_layout(margin=dict(l=20, r=20, t=30, b=20))
        return fig
    else:
        # Return empty figure if no data
        return px.pie(names=['No Data'], values=[1], title='No Data Available')
    

if __name__ == '__main__':
    app.run(debug=True)

2025-04-13 16:58:19,611 - grazioso.crud - INFO - Successfully connected to MongoDB at localhost:27017
2025-04-13 16:58:19,662 - grazioso.crud - INFO - Query returned 10000 documents
c:\Users\Brett\AppData\Local\pypoetry\Cache\virtualenvs\grazioso-salvare-ZsD8LZJq-py3.12\Lib\site-packages\dash\dash.py:582: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



2025-04-13 16:58:20,897 - grazioso.crud - INFO - Query returned 10000 documents
